# Predicting Newsletter Subscription from Player Age in a Minecraft Research Server


# Introduction:






# Methods & Results:

In [5]:
library(tidyverse)
library(repr)
library(tidymodels)
library(ggplot2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [6]:
players <- read_csv("https://raw.githubusercontent.com/calentynes/dsci_group_project/refs/heads/master/players.csv")
sessions <- read_csv("https://raw.githubusercontent.com/calentynes/dsci_group_project/refs/heads/master/sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



# Discussion:
- summarize what you found
- discuss whether this is what you expected to find
- discuss what impact could such findings have
- discuss what future questions could this lead to
  
# References

You may include references if necessary, as long as they all have a consistent citation style.

Combining two datasets together:

In [7]:
sessions <- sessions |>
  mutate(session_length = as.numeric(difftime(end_time, start_time, units = "hours")))
player_summary <- sessions |>
  group_by(hashedEmail) |>
  summarize(
    total_sessions = n(),
    avg_session_length = mean(session_length, na.rm = TRUE),
    total_play_time = sum(session_length, na.rm = TRUE))
player_data <- players |>
  left_join(player_summary, by = "hashedEmail")

In [ ]:
players_clean <- players |> filter(!is.na(subscribe))
p_age <- players_clean |>
  filter(!is.na(Age)) |>
  mutate(age_group = cut(Age, breaks = seq(5, 60, by = 5), right = TRUE)) |>
  ggplot(aes(x = age_group, fill = subscribe)) +
  geom_bar(position = "fill") +
  scale_y_continuous(labels = percent) +
  labs(
    title = "Figure 1. Subscription rate by age group",
    x = "Age group (years)",
    y = "Proportion of players"
  )
p_age

In [ ]:
players_age <- players |>
  filter(!is.na(Age),
         !is.na(subscribe)) |>
  mutate(subscribe = factor(subscribe,
                       levels = c(FALSE, TRUE),
                       labels = c("No", "Yes")))

In [ ]:
players_age |>
  summarise(
    n = n(),
    mean_age = mean(Age),
    sd_age   = sd(Age)
  )

players_age |>
  count(subscribe)

In [ ]:
set.seed(123)
players_split <- initial_split(players_age, prop = 0.8, strata = subscribe)
players_train <- training(players_split)
players_test  <- testing(players_split)

nrow(players_train); nrow(players_test)

In [ ]:
subscribe_recipe <- recipe(subscribe ~ Age, data = players_train) |>
  step_normalize(all_predictors())

subscribe_recipe

In [2]:
knn_spec <- nearest_neighbor(
    weight_func = "rectangular",
    neighbors   = tune()
  ) |>
  set_engine("kknn") |>
  set_mode("classification")

set.seed(123)
players_vfold <- vfold_cv(players_train, v = 5, strata = subscribe)

players_wflow <- workflow() |>
  add_recipe(subscribe_recipe) |>
  add_model(knn_spec)

grid_k <- tibble(neighbors = seq(1, 51, by = 2))

set.seed(123)
knn_results <- players_wflow |>
  tune_grid(resamples = players_vfold,
            grid      = grid_k) |>
  collect_metrics()

knn_results |>
  filter(.metric == "accuracy") |>
  arrange(desc(mean)) |>
  slice(1)

ERROR: Error in set_mode(set_engine(nearest_neighbor(weight_func = "rectangular", : could not find function "set_mode"


In [3]:
best_k <- knn_results |>
  filter(.metric == "accuracy") |>
  slice_max(mean, n = 1) |>
  pull(neighbors)

best_k

ERROR: Error in pull(slice_max(filter(knn_results, .metric == "accuracy"), mean, : could not find function "pull"


In [4]:
final_knn_spec <- nearest_neighbor(
    weight_func = "rectangular",
    neighbors   = best_k
  ) |>
  set_engine("kknn") |>
  set_mode("classification")

final_wflow <- workflow() |>
  add_recipe(subscribe_recipe) |>
  add_model(final_knn_spec)

final_fit <- final_wflow |>
  fit(data = players_train)

ERROR: Error in set_mode(set_engine(nearest_neighbor(weight_func = "rectangular", : could not find function "set_mode"


In [20]:
test_pred <- predict(final_fit, players_test, type = "prob") |>
  bind_cols(
    predict(final_fit, players_test),   
    players_test |> select(subscribe, Age)
  )

head(test_pred)

.pred_No,.pred_Yes,.pred_class,subscribe,Age
<dbl>,<dbl>,<fct>,<fct>,<dbl>
0.44,0.56,Yes,Yes,17
0.44,0.56,Yes,No,17
0.20,0.80,Yes,No,21
0.44,0.56,Yes,Yes,17
0.48,0.52,Yes,Yes,58
0.60,0.40,No,No,18
